In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv

# .env 로드 (상위 디렉토리에서 자동 탐색)
load_dotenv()  # 또는 load_dotenv(dotenv_path='../.env')
api_key = os.getenv('OPENAI_API_KEY')
print('API Key loaded' if api_key else 'API Key not found!')

# ===== 프롬프트 구성 (혁신-안정성 균형형 · 우수퀄리티 4/5) =====
persona = """
당신은 금융권 디지털 전환 프로젝트를 다수 성공시킨 혁신적 기술 리더입니다(경력 15년+).
최신 기술 트렌드를 적극 도입하되, 금융권의 안정성과 규제 요구사항을 완벽히 준수하는 균형잡힌 접근을 취합니다.
구체적인 설계 근거, 정량적 성과 지표, 실제 유사 사례를 바탕으로 신뢰성 있는 제안을 작성합니다.
단계적 구축 전략으로 리스크를 최소화하면서도 혁신적 가치를 제공합니다.
"""

concept = """
이번 제안은 '혁신-안정성 균형형' 컨셉입니다.
검증된 기술 스택을 기반으로 하되, 최신 UX/UI 패턴, 클라우드 네이티브 아키텍처, 자동화된 품질 관리를 적극 도입합니다.
**금융권 우수 사례 수준**의 구체적이고 달성 가능한 성과 지표를 제시하며, 단계적 개선 로드맵을 함께 제안합니다.
RFP 요구사항을 대부분 충족하고, 추가 가치를 제공하는 차별화 포인트를 명확히 제시합니다.
"""

# 우수퀄리티(4/5) 가정: RFP 요구사항 대부분 충족, 구체적 근거와 사례 제시
quality_constraints = """
[품질 요구(우수 4/5)]
- RFP 요구사항 대부분(80% 이상)을 구체적으로 제안하고, 일부는 추가 가치 제안 포함.
- 근거 수치는 구체적으로 제시(성능, 처리량, 응답시간 등)하되 측정 방법론도 간략 설명.
- 유사 사례 2~3개를 실명 또는 익명으로 제시(규모, 성과, 적용 기술 포함).
- 일정은 주요 마일스톤별로 구분하고, 조직은 핵심 역할과 책임(RACI 수준) 명시.
- 테스트 전략은 단위/통합/부하/보안 테스트별로 구분하여 기준 제시.
- 보안/컴플라이언스는 적용 표준(ISO, 금융권 가이드라인)과 구현 방안 명시.
- KPI는 단계별(구축 직후/3개월 후/6개월 후) 목표치 제시.
"""

# 금융권 평균 수준의 현실적인 수치 가이드라인
performance_guidelines = """
###금융권 우수 사례 수준의 현실적인 수치 가이드라인
다음 범위 내에서 **금융권 우수 사례 수준**의 현실적인 수치를 사용하세요:

**시스템 성능 지표**
- **서비스 가용률**: 97.5% ~ 98.0% (연간 계획된 유지보수 시간 제외, 1차년도 97.0% → 2차년도 98.0% 개선)
- **평균 응답시간**: 1.5~2초
- **최대 응답시간**: 5~6초 (복잡한 쿼리 기준)
- **동시 접속자 처리**: 2,000~3,000명
- **시간당 처리량**: 10만~15만 건
- **데이터베이스 쿼리 응답시간**: 100~200ms (일반 쿼리), 200~300ms (복잡 쿼리)

**데이터 처리 지표**
- **배치 처리 시간**: 30분~1시간 (일별 집계 기준, 병렬 처리 적용)
- **실시간 이벤트 처리 지연**: 2~5초
- **데이터 정합성**: 99~99.5% (자동 검증 프로세스 적용)

**안정화 및 운영 방안**
- **시스템 안정화 기간**: 오픈 후 2개월 (전담 안정화 팀 상주)
- **장애 등급별 대응체계**:
  - 긴급(Critical): 15분 이내 1차 대응 착수, RTO 2~3시간
  - 높음(High): 1시간 이내 대응 착수, RTO 4~6시간
  - 보통(Medium): 4시간 이내 대응 착수, RTO 12~24시간
- **백업 정책**:
  - 전체 백업: 주 1회 (주말 심야)
  - 증분 백업: 일 2회 (낮/밤)
  - 트랜잭션 로그 백업: 3시간마다
  - RPO (목표 복구 시점): 최대 3시간 이내
  - 백업 데이터 복구 테스트: 월 1회
- **무상 유지보수**: 시스템 오픈 후 12개월
- **모니터링**: 실시간 성능/오류 모니터링, 임계치 기반 알림 (Grafana/Prometheus 수준)

**품질 관리**
- **테스트 커버리지**: 코드 커버리지 70~75%
- **결함 밀도**: 1,000 라인당 2~3개 이하 (오픈 전)
- **테스트 케이스 수**: 기능당 평균 5~8개 (정상/예외 케이스 포함)

###절대 사용 금지 수치 (비현실적)
다음 수치들은 **절대 사용하지 마세요**:
- ❌ 서비스 가용률 99.0% 이상
- ❌ 평균 응답시간 1초 이하
- ❌ 데이터 정합성 100%
- ❌ "장애 복구 30분 이내 보장" 같은 절대적 시간 보장
- ❌ "무장애 시스템" 같은 비현실적 표현
- ❌ 안정화 기간 1개월 이내
- ❌ 테스트 커버리지 90% 이상

###현실적인 제안서 작성 방식
1. **측정 가능한 지표 제시**: 각 수치에 대한 측정 방법과 도구 명시
2. **단계적 개선 목표**: "1차년도 97.0% → 2차년도 98.0%" 형태로 제시
3. **전제 조건 명시**: "정상 운영 시", "계획된 유지보수 제외", "표준 부하 기준"
4. **비교 기준 제시**: "유사 금융권 시스템 대비", "현행 시스템 대비 X% 개선"
"""

# 신한투자증권 로그정보시스템 고도화 RFP 컨텍스트 요약 (필수 요구)
goal_context = """
당신은 지금 {신한투자증권 로그정보시스템 고도화} 사업의 입찰 경쟁에 참여하고 있습니다.
가. 사업명: 『신한투자증권 로그정보시스템 고도화』
나. 사업 주요 내용(안)
  (1) 사업 취지
    - 메타 차세대 프로젝트와 연계된 신규 로그를 사용자가 이해하기 쉬운 웹 UI로 조회 가능하도록 시스템 고도화
  (2) 추진 목적
    - 사용자 중심 UX로 로그 탐색·검색·필터링 경험 개선
    - 컴플라이언스 이슈 대응: 민감정보 항목 승인/권한 통제 및 감사 추적 강화
    - 그룹 권한(IT/일반직원 등)에 따른 로그 조회 제한과 차등 정보 노출
  (3) 요구 사항
    - 메타 신규 거래로그 조회(표준 스키마/메타데이터 정합성)
    - 권한/승인 관리 프로세스(요청-검토-승인-만료) 개발
    - 권한별 화면제어, 공통코드 유효값 연동, 로그 상세 화면 설계
다. 구축 기간: 2024년 10월 ~ 2024년 12월(예정)
라. 비고: 상세 범위·예산은 본 RFP 본문에서 확정 예정(현행 연계/SSO/IAM·보안 정책 준수)
"""

# 제안서 공통 구조
instructions = """
<출력 형식>
- 첫 줄: 제안서: 신한투자증권 로그정보시스템 고도화 – 혁신-안정성 균형형 제안(가상)
- 둘째 줄: 제안사: [가상 회사명], 작성일/담당자: [간략 표기]

<제안서 구조(상세형)>
1. 제안 개요(배경/목표/차별화 포인트/단계별 KPI)
2. 제안사 개요 및 유사 수행 경험(실제 사례 2~3개, 성과 지표 포함)
3. 프로젝트 수행
   3.1 기술 부문: 시스템 아키텍처(상세 구성도 설명), 기술 스택 선택 근거, 성능 목표 및 측정 방법
   3.2 테스트 전략: 단위/통합/부하/보안 테스트별 기준 및 도구
   3.3 보안 및 컴플라이언스: 적용 표준, 구현 방안, 감사 추적 구조
   3.4 일반 부문: 상세 일정(마일스톤별), 조직 구성(RACI), 방법론(Agile 등)
4. 사업 관리: 리스크 관리(주요 리스크별 완화 전략), 품질 관리(QA 프로세스), 변경 관리
5. 안정화 및 운영: 안정화 계획, 장애 대응 체계, 백업/복구 전략, 모니터링 방안
6. 결론: 핵심 가치 제안, 차별화 요소, 고객 이점

<제안서 세부 작성 방법(우수퀄리티·혁신-안정성 균형형·8항목)>
1) 문체는 전문적이면서도 명확하고, 근거 기반의 논리적 설득력을 갖춘 표현 사용.
2) RFP 요구사항 80% 이상을 구체적으로 제안하고, 일부는 추가 가치(예: AI 기반 로그 분석 제안) 포함.
3) 기술 스택 선택 시 구체적 근거 제시:
   - "Spring Boot: 엔터프라이즈 지원 우수, 보안 패치 신속, 사내 기술 스택 호환성"
   - "React: 컴포넌트 재사용성, 대규모 커뮤니티, 금융권 채택 사례 다수(예: 신한은행 모바일)"
4) 성능/가용성/보안은 구체적 수치와 측정 방법 제시 - 위 가이드라인의 **중상위** 수치 사용:
   - 가용률 97.5% (1차년도) → 98.0% (2차년도), 측정: Uptime Robot + Grafana
   - 평균 응답 1.5~2초 (단순 조회 1.5초, 복잡 쿼리 2초), 측정: APM 도구(Pinpoint)
   - 동시 접속 2,500명 처리, 근거: 부하 테스트(JMeter) 결과 기반
5) 테스트 전략은 단계별로 구분하여 제시:
   - 단위 테스트: JUnit 5, 커버리지 70% 목표
   - 통합 테스트: TestContainers, 주요 시나리오 20개
   - 부하 테스트: JMeter, 목표 TPS 50/동시 사용자 2,500명
   - 보안 테스트: OWASP ZAP, 취약점 스캔 + 침투 테스트
6) 일정/조직은 구체적으로 제시:
   - 일정: 주요 마일스톤(요구사항 확정 2주, 설계 3주, 개발 7주, 테스트 2주)
   - 조직: PM 1, 아키텍트 1, 백엔드 개발 3, 프론트 개발 2, QA 2 (RACI 매트릭스 포함)
7) 리스크 관리는 주요 리스크별로 표로 제시:
   - 리스크: 메타 시스템 연동 지연 | 영향: 중 | 확률: 중 | 완화: Mock 데이터 환경 사전 구축
   - 리스크: 성능 목표 미달성 | 영향: 고 | 확률: 저 | 완화: 캐싱 전략(Redis) + 쿼리 최적화
8) 규정·컴플라이언스는 구체적 구현 방안 제시:
   - 개인정보보호법: 민감정보 마스킹(AES-256), 접근 로그 3년 보관
   - 금융권 보안 가이드라인: 세션 타임아웃 15분, 비밀번호 정책(특수문자 포함 10자 이상)
   - 감사 추적: 모든 조회/수정 이력을 별도 감사 테이블에 저장(누가/언제/무엇을/왜)
"""

# 프롬프트 결합
prompt = f"{persona}\n{concept}\n{performance_guidelines}\n{goal_context}\n{quality_constraints}\n{instructions}"

# 최신 openai 라이브러리 방식 (openai>=1.0.0)
client = OpenAI(api_key=api_key)
response = client.chat.completions.create(
    model="gpt-5-nano",
    messages=[
        {"role": "user", "content": prompt}
    ]
)

API Key loaded


In [2]:
# 답변 출력
print(response.choices[0].message.content)

제안서: 신한투자증권 로그정보시스템 고도화 – 혁신-안정성 균형형 제안(가상)
제안사: NexWave Solutions, 작성일/담당자: 2025-10-01 / 이수현 이사

1. 제안 개요
- 배경 및 필요성
  - 메타 차세대 거래 로그와 표준 스키마 연계 필요성 증가에 따라, 사용자 친화적 UX 기반의 실시간 조회 및 강력한 컴플라이언스 제어가 요구됩니다.
  - 다중 그룹 권한(IT/일반직원) 및 민감정보 관리 강화가 필수인 상황에서, 승인-요청-만료 라이프사이클 관리와 감사 추적을 내재화한 시스템이 필요합니다.
- 목표
  - 사용자 중심 UX를 통해 로그 탐색, 검색, 필터링 경험 개선
  - 컴플라이언스 이슈 대응 강화: 민감정보 항목 승인/권한 통제 및 감사 추적 강화
  - 그룹 권한에 따른 로그 조회 제한 및 차등 정보 노출 구현
- 차별화 포인트
  - 메타 로그의 표준 스키마/메타데이터 정합성 보장으로 거버넌스 강화
  - 데이터 마스킹, 필드 단위 접근 제어, 감사 로그의 강력한 포렌식 추적
  - AI 기반 태깅/카테고라이제이션으로 검색 시나리오 자동 보강
- 단계별 KPI(범위 지향, 측정 방법 포함)
  - 구축 직후: 가용 97.0%, 평균 응답 1.8s(단순 조회 기준), 동시접속 2,500명, TPS 50,000~60,000건/시
  - 3개월: 가용 97.5%, 평균 응답 1.7s, 동시접속 2,800명, TPS 60,000~70,000건/시
  - 6개월: 가용 98.0%, 평균 응답 1.6–1.8s(복합 조회 포함), 동시접속 3,000명, TPS 80,000~100,000건/시
  - 데이터 처리: 배치 30분~1시간, 실시간 이벤트 지연 2–5초, 데이터 정합성 99.0–99.5%
  - 안정화/운영: 2개월간 전담 안정화 팀 상주
- 전제 조건
  - 정상 운영 시, 표준 유지보수 제외
  - 메타 신규 거래로그의 표준 스키마 및 연계 인터페이스 확정
  - SSO/IAM 및 보안 정책의 현행 규정 준수 유지

2.

In [3]:
import re

def markdown_to_structured_html(markdown_text):
    """
    Markdown 텍스트를 구조화된 HTML로 변환
    Upstage OCR Parser의 출력 형식과 유사한 계층 구조 생성
    """
    html_lines = ['<!DOCTYPE html>', '<html>', '<head>', 
                  '<meta charset="UTF-8">', 
                  '<title>제안서</title>',
                  '<style>',
                  'body { font-family: "Malgun Gothic", sans-serif; line-height: 1.6; margin: 20px; }',
                  'h1 { color: #2c3e50; border-bottom: 3px solid #3498db; padding-bottom: 10px; }',
                  'h2 { color: #34495e; margin-top: 30px; border-bottom: 2px solid #95a5a6; padding-bottom: 8px; }',
                  'h3 { color: #7f8c8d; margin-top: 20px; }',
                  'table { border-collapse: collapse; width: 100%; margin: 20px 0; }',
                  'th, td { border: 1px solid #ddd; padding: 12px; text-align: left; }',
                  'th { background-color: #3498db; color: white; }',
                  'ul, ol { margin: 10px 0; padding-left: 30px; }',
                  'p { margin: 10px 0; }',
                  '</style>',
                  '</head>', '<body>']
    
    lines = markdown_text.split('\n')
    in_table = False
    in_list = False
    
    for line in lines:
        line = line.strip()
        
        if not line:
            if in_list:
                html_lines.append('</ul>')
                in_list = False
            html_lines.append('<br/>')
            continue
        
        # 제목 변환
        if line.startswith('# '):
            if in_list:
                html_lines.append('</ul>')
                in_list = False
            html_lines.append(f'<h1>{line[2:]}</h1>')
        elif line.startswith('## '):
            if in_list:
                html_lines.append('</ul>')
                in_list = False
            html_lines.append(f'<h2>{line[3:]}</h2>')
        elif line.startswith('### '):
            if in_list:
                html_lines.append('</ul>')
                in_list = False
            html_lines.append(f'<h3>{line[4:]}</h3>')
        elif line.startswith('#### '):
            if in_list:
                html_lines.append('</ul>')
                in_list = False
            html_lines.append(f'<h4>{line[5:]}</h4>')
        
        # 표 감지 (| ... | 형태)
        elif '|' in line and not in_table:
            if in_list:
                html_lines.append('</ul>')
                in_list = False
            html_lines.append('<table>')
            cells = [cell.strip() for cell in line.split('|') if cell.strip()]
            html_lines.append('<tr>')
            for cell in cells:
                html_lines.append(f'<th>{cell}</th>')
            html_lines.append('</tr>')
            in_table = True
        elif '|' in line and in_table:
            if '---' in line:  # 구분선 무시
                continue
            cells = [cell.strip() for cell in line.split('|') if cell.strip()]
            html_lines.append('<tr>')
            for cell in cells:
                html_lines.append(f'<td>{cell}</td>')
            html_lines.append('</tr>')
        elif in_table and '|' not in line:
            html_lines.append('</table>')
            in_table = False
        
        # 목록 변환
        elif line.startswith('- ') or line.startswith('* '):
            if not in_list:
                html_lines.append('<ul>')
                in_list = True
            content = line[2:].strip()
            # **굵게** 변환
            content = re.sub(r'\*\*(.+?)\*\*', r'<strong>\1</strong>', content)
            html_lines.append(f'<li>{content}</li>')
        
        # 일반 텍스트
        else:
            if in_list:
                html_lines.append('</ul>')
                in_list = False
            # **굵게** 변환
            line = re.sub(r'\*\*(.+?)\*\*', r'<strong>\1</strong>', line)
            html_lines.append(f'<p>{line}</p>')
    
    if in_table:
        html_lines.append('</table>')
    if in_list:
        html_lines.append('</ul>')
    
    html_lines.extend(['</body>', '</html>'])
    
    return '\n'.join(html_lines)

# 제안서를 구조화된 HTML로 변환
proposal_text = response.choices[0].message.content
structured_html = markdown_to_structured_html(proposal_text)

# HTML 파일로 저장
output_path = 'proposal_advanced_4of5.html'
with open(output_path, 'w', encoding='utf-8') as f:
    f.write(structured_html)

print(f"구조화된 HTML 제안서가 생성되었습니다: {output_path}")
print(f"총 {len(structured_html)} 문자")
print(structured_html)

구조화된 HTML 제안서가 생성되었습니다: proposal_advanced_4of5.html
총 7808 문자
<!DOCTYPE html>
<html>
<head>
<meta charset="UTF-8">
<title>제안서</title>
<style>
body { font-family: "Malgun Gothic", sans-serif; line-height: 1.6; margin: 20px; }
h1 { color: #2c3e50; border-bottom: 3px solid #3498db; padding-bottom: 10px; }
h2 { color: #34495e; margin-top: 30px; border-bottom: 2px solid #95a5a6; padding-bottom: 8px; }
h3 { color: #7f8c8d; margin-top: 20px; }
table { border-collapse: collapse; width: 100%; margin: 20px 0; }
th, td { border: 1px solid #ddd; padding: 12px; text-align: left; }
th { background-color: #3498db; color: white; }
ul, ol { margin: 10px 0; padding-left: 30px; }
p { margin: 10px 0; }
</style>
</head>
<body>
<p>제안서: 신한투자증권 로그정보시스템 고도화 – 혁신-안정성 균형형 제안(가상)</p>
<p>제안사: NexWave Solutions, 작성일/담당자: 2025-10-01 / 이수현 이사</p>
<br/>
<p>1. 제안 개요</p>
<ul>
<li>배경 및 필요성</li>
<li>메타 차세대 거래 로그와 표준 스키마 연계 필요성 증가에 따라, 사용자 친화적 UX 기반의 실시간 조회 및 강력한 컴플라이언스 제어가 요구됩니다.</li>
<li>다중 그룹 권한(IT/일반직원) 및 민감정보 관리 강화가 필